In [1]:
import numpy as np
import pandas as pd
from pyquery import PyQuery as pq
import time
import datetime
import requests
import urllib.request
import os
import pickle
from selenium import webdriver
from lxml import etree
from selenium.webdriver.support.ui import Select
import re
from selenium.webdriver.chrome.options import Options

In [190]:
def crawl_region_pyql(region_list,url='http://search.qyer.com/all/271.html', pages=5):
    opts = Options()
    # opts.add_argument("--incognito")  # 使用無痕模式

    ua = "Mozilla/5.0 (Windows NT 10.0; WOW64; rv:53.0) Gecko/20100101 Firefox/53.0"
    opts.add_argument("user-agent={}".format(ua))         # 使用偽造的 user-agent

    prefs = {'download.default_directory': './data'}
    opts.add_experimental_option('prefs', prefs)
    browser = webdriver.Chrome('/Users/lou/Downloads/chromedriver',options=opts)
    browser.get(url)
    browser.implicitly_wait(10)
    current_window = browser.current_window_handle
    
    attraction_regions = {}
    
    for region in region_list:
        browser.find_element_by_xpath("//div[@class='q-header-search']//input[@name='keyword']").send_keys(region)
        browser.find_element_by_xpath("//button[@class='btn-search']").click()
        browser.find_element_by_xpath("//a[contains(text(),'景点美食')]").click()

        attraction_list = []
        all_windows = browser.window_handles

        # 切換 window
        for window in all_windows:
            if window != current_window:
                browser.switch_to.window(window)
        
        doc = pq(browser.current_url,encoding="utf-8")
        all_pages = int(doc('.ui_page').text().split(' ')[-2].replace('.',''))
        crawl_pages = min(all_pages,pages)
        
        # 拿attractions for the first page
        attractions = doc('.plcPoilistModules .title.fontYaHei').text().replace('\xa0','')
        attraction_list.append(attractions)
        
        for p in range(crawl_pages-1):
            time.sleep(5)
            browser.find_element_by_css_selector('.ui_page_item.ui_page_next').click()
            
            # 拿attractions
            element = browser.find_element_by_css_selector('.plcPoilistModules')
            doc = pq(element.get_attribute('innerHTML'))
            attractions = doc('.title.fontYaHei').text()#.replace('\xa0','')
            
            attraction_list.append(attractions)
            
        attraction_regions[region] = attraction_list

        # 關窗口與切換窗口
        browser.close()    
        browser.switch_to.window(current_window)
    browser.quit()
    return attraction_regions

In [186]:
region_list = ['台北','台中','台南']

In [187]:
attraction_regions = crawl_region_pyql(region_list,url='http://search.qyer.com/all/271.html', pages=5)

In [193]:
attraction_regions

{'台北': ['台北101大楼  Taipei 101 Mall 台北故宫博物院  National Palace Museum 西门町 中正纪念堂  Chiang Kai-Shek Memorial Hall 国父纪念馆  National Dr. Sun Yat-sen Memorial Hall 士林夜市  Shilin Night Market Taipei 士林官邸  Shilin Official Residence 台北总统府  Presidential Building 阿宗面线 猫空站  Maokong 红楼创意市集 龙山寺  Longshan Temple 阳明山国家公园  Yangmingshan National Park 101觀景台  101 Observatory 北投温泉博物馆',
  '台北101大楼 \xa0\xa0 Taipei 101 Mall 台北故宫博物院 \xa0\xa0 National Palace Museum 西门町 中正纪念堂 \xa0\xa0 Chiang Kai-Shek Memorial Hall 国父纪念馆 \xa0\xa0 National Dr. Sun Yat-sen Memorial Hall 士林夜市 \xa0\xa0 Shilin Night Market Taipei 士林官邸 \xa0\xa0 Shilin Official Residence 台北总统府 \xa0\xa0 Presidential Building 阿宗面线 猫空站 \xa0\xa0 Maokong 红楼创意市集 龙山寺 \xa0\xa0 Longshan Temple 阳明山国家公园 \xa0\xa0 Yangmingshan National Park 101觀景台 \xa0\xa0 101 Observatory 北投温泉博物馆',
  '101觀景台\xa0\xa0101 Observatory 北投地热谷\xa0\xa0Hell Valley 阿宗面线 饶河街观光夜市 悠游卡\xa0\xa0EasyCard 宁夏观光夜市 士林夜市\xa0\xa0Shilin Night Market Taipei 永康街与青田街\xa0\xa0YongKang &QingTian Street 诚品书店(敦南店)\xa0\

### crawl_tripadvisior

In [60]:
def crawl_region_trip(region_list,url_temp, pages=5):
    opts = Options()
    opts.add_argument("--incognito")  # 使用無痕模式

    ua = "Mozilla/5.0 (Windows NT 10.0; WOW64; rv:53.0) Gecko/20100101 Firefox/53.0"
    opts.add_argument("user-agent={}".format(ua))         # 使用偽造的 user-agent

    prefs = {'download.default_directory': './data'}
    opts.add_experimental_option('prefs', prefs)
    browser = webdriver.Chrome('/Users/lou/Downloads/chromedriver',options=opts)
#     current_window = browser.current_window_handle
    
    attraction_regions = {}
    headers = {'x-requested-with': 'XMLHttpRequest'}
    
    for i,region in enumerate(region_list):
        url = url_temp.replace('location',region)
        
        attraction_list = []
        
        browser.get(url)
        time.sleep(3)
#         browser.find_element_by_css_selector('.search-filter.ui_tab').click()
#         browser.find_element_by_xpath("//div[@id='search-filters']//a[contains(@class,'')][contains(text(),'觀光')]").click()
        try:
            browser.find_element_by_css_selector(".ui_link.show-text").click() # 更多
        except:
            print('No more!!')
        time.sleep(3)
        
        # Get attractions
        doc = pq(browser.page_source)
        tags = doc('.thumbnail-overlay-tag')
        title = doc('.result-title')
#         break
#     return doc,tags, title
        for j,t in enumerate(title.items()):
#             print(d.text())
            attraction_list.append((t.text(),tags[j].text))
        
        all_pages = int(doc('.pageNum ').text().split(' ')[-1])
        crawl_pages = min(all_pages,pages)
        
        for p in range(crawl_pages-1):
            browser.find_element_by_xpath("//a[contains(@class,'ui_button nav next primary')]").click()
            time.sleep(5)
#             browser.find_element_by_css_selector('search-filter.ui_tab').click()
#             browser.find_element_by_css_selector(".ui_link.show-text").click() # 更多
            
            # 拿attractions
            doc = pq(browser.page_source)

            tags = doc('.thumbnail-overlay-tag')
            title = doc('.result-title')
            for j,d in enumerate(title.items()):
                try:
                    attraction_list.append((d.text(),tags[j].text))
                except:
                    attraction_list.append((d.text(),''))

        attraction_regions[region] = attraction_list

        # 關窗口與切換窗口
#         browser.close()    
#         browser.switch_to.window(current_window)
    browser.quit()
    return attraction_regions

In [61]:
url = 'https://www.tripadvisor.com.tw/Search?q=location&ssrc=A&rf=5'
attraction_regions_t = crawl_region_trip(['台北','台中','台南'],url, pages=3)

No more!!
No more!!
No more!!


In [62]:
attraction_regions_t

{'台北': [('臺北捷運', '公共交通系統'),
  ('台北101', '建築物'),
  ('桃園機場捷運線', '公共交通系統'),
  ('台北101觀景台', '建築物'),
  ('象山自然步道', '景點和地標'),
  ('台北地下街', '購物中心'),
  ('微風台北車站', '特色和禮品商店'),
  ('臺北101跨年煙火', '景點'),
  ('台北車站', '公共交通系統'),
  ('西門町', '地區'),
  ('貓空纜車', '電車'),
  ('台北小巨蛋', '競技場和體育場館'),
  ('台北賓館', '建築物'),
  ('陽明山國家公園', '史跡徒步區'),
  ('MyTaiwanTour Travel Services', '潛水和浮潛'),
  ('京站時尚廣場', '購物中心'),
  ('TourMeAway - Free Walking Tour In Taipei', '觀光遊覽'),
  ('台北國際書展', '景點'),
  ('龍山寺', '宗教景點'),
  ('台北車站 (桃園捷運)', '公共交通系統'),
  ('台北植物園', '花園'),
  ('大安森林公園', '公園'),
  ('新光三越 台北站前店', '特色和禮品商店'),
  ('台北孔廟', '古蹟'),
  ('貓空山', '電車'),
  ('迪化街', '特色和禮品商店'),
  ('北投溫泉', '溫泉和間歇泉'),
  ('台北國際會議中心', '會議中心'),
  ('台北當代藝術館', '藝術博物館'),
  ('新光三越 台北信義新天地', '特色和禮品商店'),
  ('台灣高鐵', '公共交通系統'),
  ('桃園捷運', '公共交通系統'),
  ('國立故宮博物院', '歷史博物館'),
  ('國光客運', '公共交通系統'),
  ('士林觀光夜市', '跳蚤市場和街頭市集'),
  ('二二八和平紀念公園', '古蹟'),
  ('饒河街觀光夜市', '跳蚤市場和街頭市集'),
  ('中正紀念堂', '紀念碑和雕像'),
  ('來去福爾摩沙', '觀光遊覽'),
  ('台北探索館', '特色博物館'),
  ('Taipei Eats', '步行遊覽'),
  ('國父紀念

### Test